In [1]:
import numpy as np
import pandas as pd

In [2]:
def split_ranges(range1, range2):
    """Split up the ranges to find the overlaps

    range1 and range2 should be sorted

    returns non-overlapping ranges, overlapping ranges
    """
    # go through the different cases

    # No overlap
    if min(range1) > max(range2):
        return range1, []
    if max(range1) < min(range2):
        return range1, []
    # perfect overlap
    if range1 == range2:
        return [], range2

    # range 1 completely on the inside
    if (min(range1) >= min(range2)) & (max(range1) <= max(range2)):
        return [], range1
    
    # partial overlap
    if  range2[0] <= range1[1] <= range2[1]:
        return [range1[0], range2[0]-1], [range2[0], range1[1]]
    if range2[0] <= range1[0] <= range2[1]:
        return [range2[1]+1, range1[1]], [range1[0], range2[1]]

    # range2 completely on the inside
    if (min(range2) >= min(range1)) & (max(range2) <= max(range1)):
        return [[range1[0],range2[0]-1], [range2[1]+1, range1[1]]], range2


In [3]:
class a_to_b_map(object):
    def __init__(self, dest_starts, source_starts, range_lengths):
        self.dest_starts = dest_starts
        self.source_starts = source_starts
        self.range_lengths = range_lengths

        self.source_ranges = []
        self.destination_ranges = []
        self.offsets = []
        for ds, ss, rl in zip(self.dest_starts, self.source_starts, self.range_lengths):
            self.source_ranges.append([ss, ss+rl-1])
            self.destination_ranges.append([ds, ds+rl-1])
            self.offsets.append(ds - ss)
    
    def __call__(self, sources):
        """sources should be a list of min,max pairs
        """

        results = []
        
        # Now need sources to be a list of pairs. 
        for source in sources:
            not_moved = source.copy()
            for source_range, offset in zip(self.source_ranges, self.offsets):
                no_dest, to_dest = split_ranges(source, source_range)
                if len(to_dest) > 0:
                    dest = [to_dest[0]+offset, to_dest[1]+offset]
                    results.append(dest)
                    # find ranges that have not been moved
                    no_overlap, overlap = split_ranges(not_moved, to_dest)
                    not_moved = no_overlap
            if len(not_moved) > 0:
                results.append(not_moved)
            if np.min(results) < 0:
                import pdb ; pdb.set_trace()
        return results

In [4]:
def numbers_to_map(in_list):
    dest_starts = []
    source_starts = []
    range_lengths = []
    for l in in_list:
        dest_starts.append(int(l[0]))
        source_starts.append(int(l[1]))
        range_lengths.append(int(l[2]))
    new_map = a_to_b_map(dest_starts, source_starts, range_lengths)
    return new_map

def read_and_make(filename):
    numbers = None
    maplist = []
    with open(filename) as fp:
        for line in fp:
            if 'seeds:' in line:
                seeds = line.replace('seeds: ', '').split(' ')
            elif len(line.strip()) == 0:
                if numbers is not None:
                    maplist.append(numbers_to_map(numbers))
                numbers = []
            elif ":" in line:
                pass
            else:
                numbers.append(line.split(' '))
        
    seeds = [int(s.strip()) for s in seeds]
    return seeds, maplist

In [5]:
seeds, maplist = read_and_make('test_data/test.dat')
#seeds, maplist = read_and_make('input')

In [6]:
seeds

[79, 14, 55, 13]

In [7]:
#locations = []
#for val in seeds:
#    for func in maplist:
#        val = func(val)
#    locations.append(val)

In [8]:
#maplist[0].source_ranges


In [9]:
#maplist[0].offsets

In [10]:
actual_seeds = []
for seed_start, length in zip(seeds[::2], seeds[1::2]):
    actual_seeds.append([seed_start, seed_start+length-1])

In [11]:
actual_seeds

[[79, 92], [55, 67]]

In [12]:
# This looks good
i=13
val = [[13,13], [14,14]]
for func in maplist:
    val = func(val)
val

[[35, 35], [43, 43]]

In [13]:
val = [[10,14]]
for func in maplist:
    val = func(val)
val

[[43, 43], [32, 35]]

In [14]:
maplist[1]([[14,15]])

[[0, 0], [53, 53]]

In [15]:

val = actual_seeds
for func in maplist:
    val = func(val)


In [16]:
val

[[60, 60], [46, 55], [82, 84], [86, 89], [94, 96], [56, 59], [97, 98]]

In [17]:
np.min(val)



46

In [18]:
i=82
val = [[i,i], [i+2, 1+2]]
for func in maplist:
    val = func(val)
print(i, val)

TypeError: '>' not supported between instances of 'list' and 'int'